# MS4610 Introduction to Data Analytics Final Project

# Amex Classification problem

## (A) Importing the datasets and libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install optuna

In [ ]:
# Data Visualization Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# ML Models
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier

# Machine Learning Preprocessing Libraries
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, balanced_accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

from sklearn.preprocessing import OneHotEncoder

# Stastical analysis libraries
from scipy.stats import probplot, chisquare
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from lightgbm import LGBMClassifier
from scipy.stats import skew
from sklearn.preprocessing import StandardScaler

# For hyperparamater tuning
import optuna

# Path to datasets
train_path = "/content/drive/MyDrive/IDA Group Project/Project Details/Training Data_2021.csv"
test_path = "/content/drive/MyDrive/IDA Group Project/Project Details/Test Data_2021.csv"

# Importing the datasets
train_data = pd.read_csv(train_path, low_memory=False)    # low_memory = False is to ensure that there are no mixed data types
test_data = pd.read_csv(test_path, low_memory=False)

In [ ]:
# Training data

print(train_data.shape)
train_data.head()

(83000, 49)


,application_key,mvar1,mvar2,mvar3,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,mvar10,mvar11,mvar12,mvar13,mvar14,mvar15,mvar16,mvar17,mvar18,mvar19,mvar20,mvar21,mvar22,mvar23,mvar24,mvar25,mvar26,mvar27,mvar28,mvar29,mvar30,mvar31,mvar32,mvar33,mvar34,mvar35,mvar36,mvar37,mvar38,mvar39,mvar40,mvar41,mvar42,mvar43,mvar44,mvar45,mvar46,mvar47,default_ind
0,230032,1696,1.6541,0.000,0.0,0.0,0,6015,322,40369,18414,missing,6423,3067,123875,missing,1,1,1,3,3,94.78,8987.18,NaN,72.25,1462,4532,2890,61,4532,1095,3376,625,1.1667,1,0,8,10,4,1,73.78,82.547,0.08696,10,0.63899,na,0,C,0
1,230033,1846,0.8095,0.000,0.0,0.0,102,7532,3171,18234,13664,missing,765,1931,42613,missing,0,0,0,0,0,74.25,953.06,953.06,4.80,1028,2099,2099,30386,2281,missing,2251,169,0.4167,0,0,8,0,2,0,99.129,missing,0,13,0.63836,na,na,L,1
2,230034,1745,0.4001,0.000,0.0,0.0,missing,2536,missing,missing,2536,missing,missing,missing,76109,missing,na,na,na,0,0,NaN,NaN,NaN,NaN,missing,missing,missing,669,4623,3772,missing,missing,25.0833,0,na,1,na,1,0,missing,29.29,0,1,1.00000,na,0,C,1
3,230035,1739,0.2193,0.000,0.0,0.0,1982,26440,4955,20316,37013,missing,0,0,84235,missing,0,0,0,0,0,NaN,0.00,NaN,0.00,1308,2525,791,91,5992,missing,3741,215,10.3333,0,0,3,3,2,0,96.272,missing,0.15385,3,0.53241,0,0,L,0
4,230036,1787,0.0118,0.225,0.0,0.0,5451,5494,5494,7987,4696,missing,2257,27815,123875,524848,0,0,0,0,0,20.51,796.67,NaN,28.72,801,2281,2281,487,2707,missing,1947,158,0.9167,0,0,2,3,2,0,115.019,missing,0,1,0.92665,na,na,L,0


In [ ]:
# Test data 

print(test_data.shape)
test_data.head()

(47000, 48)


,application_key,mvar1,mvar2,mvar3,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,mvar10,mvar11,mvar12,mvar13,mvar14,mvar15,mvar16,mvar17,mvar18,mvar19,mvar20,mvar21,mvar22,mvar23,mvar24,mvar25,mvar26,mvar27,mvar28,mvar29,mvar30,mvar31,mvar32,mvar33,mvar34,mvar35,mvar36,mvar37,mvar38,mvar39,mvar40,mvar41,mvar42,mvar43,mvar44,mvar45,mvar46,mvar47
0,578069,1719,0.6174,8.623,0.0,0.0,258,258,258,10729,307,1141,40,0,49550,83387,0,0,0,0,1,NaN,6125.00,1333.33,13.33,1399,1734,122,0,3650,3437,missing,184,5.1667,0,0,3,3,2,0,missing,missing,0.54545,2,0.91837,0,0,C
1,578070,1795,0.2051,0.000,0.0,0.0,1685,12711,8913,80519,18099,missing,3457,455,198200,458833,0,0,0,0,0,25.050,2385.71,2385.71,0.00,1440,2464,2220,760,7330,2525,missing,852,3.0833,0,0,2,13,4,0,missing,missing,0.17241,4,0.94563,0,0,C
2,578071,1742,0.5082,0.000,0.0,0.0,1185,8954,8954,1189,1185,missing,3028,1453,122884,missing,0,0,0,0,0,20.340,33.33,33.33,0.33,2716,5384,5384,669,14478,3711,missing,625,0.5833,0,0,1,3,1,0,missing,missing,0.64706,1,0.97054,0,0,C
3,578072,1685,0.2595,25.409,0.0,0.0,missing,3354,missing,missing,3354,4231,missing,missing,118920,375589,na,na,na,0,0,NaN,NaN,NaN,NaN,missing,missing,missing,0,2373,missing,missing,missing,0.4167,0,na,6,na,2,0,missing,missing,0.85714,1,1.00000,na,0,C
4,578073,1666,1.2678,0.000,0.0,0.0,570,570,570,missing,570,missing,74,missing,42613,missing,0,0,0,0,0,11.538,1153.85,1153.85,NaN,0,0,0,0,1551,30,missing,0,0.1667,0,0,1,0,2,0,missing,101.61,missing,0,0.99617,na,0,L


## (B) Data Cleaning and Preprocessing of Training and Test dataset

### Checking for missing values and converting all missing values to NaN

We notice that all missing values are not given the value 'NaN'. Some of them are named with different placeholders like 'missing' and 'na'. We need to find out all such placeholders and replace them with NaN.

Steps
1. Finding out all placeholders that represent missing values (from both train and test datasets)
2. Replacing all such placeholders with 'NaN' (for both train and test datasets)

**Note**: Here, we assume that the missing value placeholder information (ie the knowledge of whether the missing value was named as 'missing', 'na' or NaN) is not useful for our prediction. It is with this assumption that we perform step 2.

**Step 1**

In [ ]:
# This function checks whether a string is a float-type or not
def is_float(string):
  try:
      float(string)
      return True
  except ValueError:
      return False

# This function extracts the unique non-numerical values from all columns of a dataset
def extract_unique_non_numerics(data):
  A  = data.values.reshape(-1)
  unique_non_numerics = set([a for a in A if not is_float(str(a))])
  return unique_non_numerics


print("Unique non-numerical values in train dataset = ",extract_unique_non_numerics(train_data) )
print("Unique non-numerical values in test dataset = ",extract_unique_non_numerics(test_data) )

Unique non-numerical values in train dataset =  {'C', 'L', 'na', 'missing'}
Unique non-numerical values in test dataset =  {'L', 'na', 'C', 'missing'}


By inspection, C and L are categories of a categorical variable, mvar47. 'missing' and 'na' are the placeholders for missing values

**Step 2**

In [ ]:
missing_map = {'missing': np.nan, 'na': np.nan}
train_data_with_NaN = train_data.replace(missing_map)
test_data_with_NaN = test_data.replace(missing_map)

### Label encoding

We could check which columns need encoding by inspecting train and test datasets

In [ ]:
train_data_with_NaN.head()

,application_key,mvar1,mvar2,mvar3,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,mvar10,mvar11,mvar12,mvar13,mvar14,mvar15,mvar16,mvar17,mvar18,mvar19,mvar20,mvar21,mvar22,mvar23,mvar24,mvar25,mvar26,mvar27,mvar28,mvar29,mvar30,mvar31,mvar32,mvar33,mvar34,mvar35,mvar36,mvar37,mvar38,mvar39,mvar40,mvar41,mvar42,mvar43,mvar44,mvar45,mvar46,mvar47,default_ind
0,230032,1696,1.6541,0.000,0.0,0.0,0,6015,322,40369,18414,NaN,6423,3067,123875,NaN,1,1,1,3,3,94.78,8987.18,NaN,72.25,1462,4532,2890,61,4532,1095,3376,625,1.1667,1,0,8,10,4,1,73.78,82.547,0.08696,10,0.63899,NaN,0,C,0
1,230033,1846,0.8095,0.000,0.0,0.0,102,7532,3171,18234,13664,NaN,765,1931,42613,NaN,0,0,0,0,0,74.25,953.06,953.06,4.80,1028,2099,2099,30386,2281,NaN,2251,169,0.4167,0,0,8,0,2,0,99.129,NaN,0,13,0.63836,NaN,NaN,L,1
2,230034,1745,0.4001,0.000,0.0,0.0,NaN,2536,NaN,NaN,2536,NaN,NaN,NaN,76109,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,669,4623,3772,NaN,NaN,25.0833,0,NaN,1,NaN,1,0,NaN,29.29,0,1,1.00000,NaN,0,C,1
3,230035,1739,0.2193,0.000,0.0,0.0,1982,26440,4955,20316,37013,NaN,0,0,84235,NaN,0,0,0,0,0,NaN,0.00,NaN,0.00,1308,2525,791,91,5992,NaN,3741,215,10.3333,0,0,3,3,2,0,96.272,NaN,0.15385,3,0.53241,0,0,L,0
4,230036,1787,0.0118,0.225,0.0,0.0,5451,5494,5494,7987,4696,NaN,2257,27815,123875,524848,0,0,0,0,0,20.51,796.67,NaN,28.72,801,2281,2281,487,2707,NaN,1947,158,0.9167,0,0,2,3,2,0,115.019,NaN,0,1,0.92665,NaN,NaN,L,0


In [ ]:
test_data_with_NaN.head()

,application_key,mvar1,mvar2,mvar3,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,mvar10,mvar11,mvar12,mvar13,mvar14,mvar15,mvar16,mvar17,mvar18,mvar19,mvar20,mvar21,mvar22,mvar23,mvar24,mvar25,mvar26,mvar27,mvar28,mvar29,mvar30,mvar31,mvar32,mvar33,mvar34,mvar35,mvar36,mvar37,mvar38,mvar39,mvar40,mvar41,mvar42,mvar43,mvar44,mvar45,mvar46,mvar47
0,578069,1719,0.6174,8.623,0.0,0.0,258,258,258,10729,307,1141,40,0,49550,83387,0,0,0,0,1,NaN,6125.00,1333.33,13.33,1399,1734,122,0,3650,3437,NaN,184,5.1667,0,0,3,3,2,0,NaN,NaN,0.54545,2,0.91837,0,0,C
1,578070,1795,0.2051,0.000,0.0,0.0,1685,12711,8913,80519,18099,NaN,3457,455,198200,458833,0,0,0,0,0,25.050,2385.71,2385.71,0.00,1440,2464,2220,760,7330,2525,NaN,852,3.0833,0,0,2,13,4,0,NaN,NaN,0.17241,4,0.94563,0,0,C
2,578071,1742,0.5082,0.000,0.0,0.0,1185,8954,8954,1189,1185,NaN,3028,1453,122884,NaN,0,0,0,0,0,20.340,33.33,33.33,0.33,2716,5384,5384,669,14478,3711,NaN,625,0.5833,0,0,1,3,1,0,NaN,NaN,0.64706,1,0.97054,0,0,C
3,578072,1685,0.2595,25.409,0.0,0.0,NaN,3354,NaN,NaN,3354,4231,NaN,NaN,118920,375589,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2373,NaN,NaN,NaN,0.4167,0,NaN,6,NaN,2,0,NaN,NaN,0.85714,1,1.00000,NaN,0,C
4,578073,1666,1.2678,0.000,0.0,0.0,570,570,570,NaN,570,NaN,74,NaN,42613,NaN,0,0,0,0,0,11.538,1153.85,1153.85,NaN,0,0,0,0,1551,30,NaN,0,0.1667,0,0,1,0,2,0,NaN,101.61,NaN,0,0.99617,NaN,0,L


By inspection, we can find that all columns except mvar47 are numerical and does not need encoding. Since mvar47 contains only C and L as categories, we can use a Label encoder

In [ ]:
le = LabelEncoder()
train_data_with_NaN[['mvar47']] = le.fit_transform(train_data_with_NaN[['mvar47']])
test_data_with_NaN[['mvar47']] = le.transform(test_data_with_NaN[['mvar47']])

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



###  Datatype correction

Now that all the columns are numerical, we can convert all of them to float datatype for uniformity

(Except application key which is just an id and default_index which is our classification target)

In [ ]:
train_data_with_NaN = train_data_with_NaN.astype('float')
test_data_with_NaN = test_data_with_NaN.astype('float')

train_data_with_NaN['application_key'] = train_data_with_NaN.application_key.astype('int')
test_data_with_NaN['application_key'] = test_data_with_NaN.application_key.astype('int')

train_data_with_NaN['default_ind'] = train_data_with_NaN.default_ind.astype('int')

### Checking for class imbalance in the training data

In [ ]:
train_data_target_split= train_data_with_NaN.default_ind.value_counts()
print(train_data_target_split)

0    59145
1    23855
Name: default_ind, dtype: int64


We can see that there is some level of class imbalance in the training data

**We need to use imbalance correction hyperparameter for XGBoost and LightGBM (this variable will be used later during model training)**

In [ ]:
scale_pos_weight_data = train_data_target_split.values.astype('float')[0]/train_data_target_split.values.astype('float')[1]
print(scale_pos_weight_data)

2.479354433032907


### Dealing with missing values: Imputing the others with the mean of the column

We impute the missing values with the mean of the column it belongs to

In [ ]:
# print("Train dataset shape before dropping rows and columns: ",train_data_with_NaN.shape)

# # Dropping columns
# threshold_columns = train_data_with_NaN.shape[0] * .7
# columns1 = train_data_with_NaN.columns
# train_data_with_NaN = train_data_with_NaN.dropna(thresh=threshold_columns, axis=1)  # dropping poor columns of train data
# columns2 = train_data_with_NaN.columns
# dropped_columns = set(columns1)-set(columns2)
# test_data_with_NaN = test_data_with_NaN.drop(dropped_columns,axis=1)                # dropping corresponding columns of test data 

# # Dropping rows
# threshold_rows = train_data_with_NaN.shape[1] * .7
# train_data_with_NaN = train_data_with_NaN.dropna(thresh=threshold_rows, axis=0)  # dropping poor rows of train data

# print("Train dataset shape after dropping rows and columns: ",train_data_with_NaN.shape)

# Imputing with mean
train_data = train_data_with_NaN.fillna(train_data_with_NaN.mean())
test_data = test_data_with_NaN.fillna(test_data_with_NaN.mean())


### Final input columns and output columns ready for training

In [ ]:
X_train = train_data.drop(['default_ind'],axis=1)
Y_train=train_data.default_ind.values

## EDA: (Not completed)

## (C) Model Selection

### We use crossvalidation (on the scaled dataset) to find the best model 

#####(We are not sure which scoring criteria is used in the leaderboard. We arbitrarily used f1 score in this case)

Note: We use stratified k fold cross validation since it is an imbalanced dataset



### Scaling and setting up the stratified k fold cross validation 

In [ ]:
# Scaling X_train
X_train_unscaled = X_train  # saving the unscaled for later use
scaler = MinMaxScaler();
X_train = scaler.fit_transform(X_train_unscaled);

print("Average f1 scores in cross validation: \n")
k_fold = StratifiedKFold(n_splits=5, shuffle=True)  # We use stratified k fold cross validation since it is an imbalanced dataset

Average f1 scores in cross validation: 



### 1: Trying out different models individually
 We have tried almost all models taught in class (except SupportVectorClassifier, which is very time consuming)




In [ ]:

# Logistic Regression
model = LogisticRegression(max_iter=1000)  # Doesn't converge unless we increase max iterations
model_f1scores = cross_val_score(estimator = model, X = X_train, y = Y_train, cv = k_fold, scoring = 'f1')
print("Logistic Regression: ",model_f1scores.mean())

# KNN
model = KNeighborsClassifier()
model_f1scores = cross_val_score(estimator = model, X = X_train, y = Y_train, cv = k_fold, scoring = 'f1')
print("KNN: ",model_f1scores.mean())

# DecisionTree
model = DecisionTreeClassifier()
model_f1scores = cross_val_score(estimator = model, X = X_train, y = Y_train, cv = k_fold, scoring = 'f1')
print("Decision Tree: ",model_f1scores.mean())

# RandomForest
model = RandomForestClassifier()
model_f1scores = cross_val_score(estimator = model, X = X_train, y = Y_train, cv = k_fold, scoring = 'f1')
print("Random Forest: ",model_f1scores.mean())

# Bagging
model = BaggingClassifier()
model_f1scores = cross_val_score(estimator = model, X = X_train, y = Y_train, cv = k_fold, scoring = 'f1')
print("Bagging: ",model_f1scores.mean())

# LGBMClassifier
model = LGBMClassifier(scale_pos_weight=scale_pos_weight_data)
model_f1scores = cross_val_score(estimator = model, X = X_train, y = Y_train, cv = k_fold, scoring = 'f1')
print("LGBM: ",model_f1scores.mean())

# XGBoost
model = XGBClassifier(scale_pos_weight=scale_pos_weight_data)
model_f1scores = cross_val_score(estimator = model, X = X_train, y = Y_train, cv = k_fold, scoring = 'f1')
print("XGBoost: ",model_f1scores.mean())

# AdaBoost
model = AdaBoostClassifier()
model_f1scores = cross_val_score(estimator = model, X = X_train, y = Y_train, cv = k_fold, scoring = 'f1')
print("AdaBoost: ",model_f1scores.mean())

Logistic Regression:  0.47986853737210433
KNN:  0.4502791854895934
Decision Tree:  0.45018836100080034
Random Forest:  0.5014614095399285
Bagging:  0.4658905386083025
LGBM:  0.5909863763158778
XGBoost:  0.588793027309474
AdaBoost:  0.49773221056003275


### Result: LGBM and XGBoost works the best

### 2: Voting classifier of LGBM and XGBoost (Both soft and hard voting are tried)

In [ ]:
estimators = []

model_1 = LGBMClassifier(scale_pos_weight=scale_pos_weight_data)
estimators.append(('lgbm', model_1))

model_2 =XGBClassifier(scale_pos_weight=scale_pos_weight_data)
estimators.append(('xgb', model_2))

# Voting Classifier (Soft voting)
model = VotingClassifier(estimators, voting='soft')
model_f1scores = cross_val_score(estimator = model, X = X_train, y = Y_train, cv = k_fold, scoring = 'f1')
print("Soft Voting of LGBM and XGBoost: ",model_f1scores.mean())

# Voting Classifier (Hard voting)
model = VotingClassifier(estimators, voting='hard')
model_f1scores = cross_val_score(estimator = model, X = X_train, y = Y_train, cv = k_fold, scoring = 'f1')
print("Hard Voting of LGBM and XGBoost: ",model_f1scores.mean())

Soft Voting of LGBM and XGBoost:  0.5909822243942535
Hard Voting of LGBM and XGBoost:  0.5914601363747639


### Results

The following models are selected (all four have almost same accuracy)

1. Light Gradient Boosted Machine (LGBM)
2. XGBoost
3. Soft Voting of 1 and 2
4. Hard Voting of 1 and 2

Default hyperparameters were given till now
(except scale_pos_weight which was assigned imbalance correction factor taken from the training dataset: Refer the portion on checking imbalance in (B))

## (D) Hyperparameter Tuning

From Model selection we have XGBoost, LightGBM and their voting (soft and hard) as the best models.

**We perform tuning only for hyperparameters in LightGBM**, (Since cross validation for XGBoost is time consuming)

### Scaling and setting up the stratified k fold cross validation 

In [ ]:
# Scaling X_train
scaler = MinMaxScaler();
X_train = scaler.fit_transform(X_train_unscaled);

k_fold = StratifiedKFold(n_splits=5, shuffle=True)  # We use stratified k fold cross validation since it is an imbalanced dataset

### 1: Hyperparameter tuning for LightGBM

(tune most hyperparameters of LightGBM except scale_pos_weight(which is given the same value as before))

In [ ]:
def objective_LightGBM(trial):
      n_estimators_values = trial.suggest_int('n_estimators', 10, 1000)
      max_depth_values = trial.suggest_int('max_depth', 1, 100)
      learning_rate_values = trial.suggest_float('learning_rate', 0.01, 0.3)
      subsample_values = trial.suggest_float('subsample', 0.2, 1)
      colsample_bytree_values = trial.suggest_float('colsample_bytree', 0.2, 1)
      clf = LGBMClassifier(scale_pos_weight=scale_pos_weight_data, 
                          n_estimators=n_estimators_values,
                          max_depth=max_depth_values,
                          learning_rate=learning_rate_values,
                          subsample=subsample_values,
                          colsample_bytree=colsample_bytree_values)
      model_f1scores = cross_val_score(estimator = clf, X = X_train, y = Y_train, cv = k_fold, scoring = 'f1')
      return model_f1scores.mean()


study = optuna.create_study(direction='maximize')
study.optimize(objective_LightGBM, n_trials=100)

trial = study.best_trial
print('Best F1 score: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2021-12-10 05:46:55,803] A new study created in memory with name: no-name-5bd40cde-7f41-4a79-9064-b0da8a9d28f3
[I 2021-12-10 05:47:38,662] Trial 0 finished with value: 0.5858901695709269 and parameters: {'n_estimators': 760, 'max_depth': 93, 'learning_rate': 0.07453924112489263, 'subsample': 0.5985016910548367, 'colsample_bytree': 0.4978558810958312}. Best is trial 0 with value: 0.5858901695709269.
[I 2021-12-10 05:48:29,554] Trial 1 finished with value: 0.5685601197601337 and parameters: {'n_estimators': 756, 'max_depth': 32, 'learning_rate': 0.16561275792170393, 'subsample': 0.9678121860159825, 'colsample_bytree': 0.769384480726603}. Best is trial 0 with value: 0.5858901695709269.
[I 2021-12-10 05:49:08,927] Trial 2 finished with value: 0.5909766205276151 and parameters: {'n_estimators': 444, 'max_depth': 17, 'learning_rate': 0.04687531646866458, 'subsample': 0.5929496646741004, 'colsample_bytree': 0.7908704158098248}. Best is trial 2 with value: 0.5909766205276151.
[I 2021-12-10 

Best F1 score: 0.595494332412956
Best hyperparameters: {'n_estimators': 760, 'max_depth': 85, 'learning_rate': 0.02935086260767057, 'subsample': 0.3792225838646886, 'colsample_bytree': 0.21433910546560286}


### 2: Hyperparameter tuning for XGBoost is skipped since it is time consuming

### 3: Hyperparameter tuning for LightGBM + XGBoost (Soft Voting)

(tune most hyperparameters of LightGBM except scale_pos_weight(which is given the same value as before))


In [ ]:
def objective_SoftVoting(trial):
      n_estimators_values = trial.suggest_int('n_estimators', 10, 1000)
      max_depth_values = trial.suggest_int('max_depth', 1, 100)
      learning_rate_values = trial.suggest_float('learning_rate', 0.01, 0.3)
      subsample_values = trial.suggest_float('subsample', 0.2, 1)
      colsample_bytree_values = trial.suggest_float('colsample_bytree', 0.2, 1)
      clf1 = LGBMClassifier(scale_pos_weight=scale_pos_weight_data, 
                          n_estimators=n_estimators_values,
                          max_depth=max_depth_values,
                          learning_rate=learning_rate_values,
                          subsample=subsample_values,
                          colsample_bytree=colsample_bytree_values)
      clf2 = XGBClassifier(scale_pos_weight=scale_pos_weight_data)
      estimators=[]
      estimators.append(('lgbm', clf1))
      estimators.append(('xgb', clf2))
      clf = VotingClassifier(estimators, voting='soft')
      model_f1scores = cross_val_score(estimator = clf, X = X_train, y = Y_train, cv = k_fold, scoring = 'f1')
      return model_f1scores.mean()


study = optuna.create_study(direction='maximize')
study.optimize(objective_LightGBM, n_trials=100)

trial = study.best_trial
print('Best F1 score: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2021-12-10 06:36:40,141] A new study created in memory with name: no-name-4a1b40ce-6333-4e32-a11e-1ecb5128ea2a
[I 2021-12-10 06:37:14,073] Trial 0 finished with value: 0.5828185493519413 and parameters: {'n_estimators': 411, 'max_depth': 46, 'learning_rate': 0.13076747679294923, 'subsample': 0.4085950844309596, 'colsample_bytree': 0.8629935395210719}. Best is trial 0 with value: 0.5828185493519413.
[I 2021-12-10 06:37:32,336] Trial 1 finished with value: 0.5796852515400689 and parameters: {'n_estimators': 453, 'max_depth': 86, 'learning_rate': 0.1806719947440587, 'subsample': 0.20457680719172658, 'colsample_bytree': 0.23884414728756811}. Best is trial 0 with value: 0.5828185493519413.
[I 2021-12-10 06:38:45,633] Trial 2 finished with value: 0.5846899721743519 and parameters: {'n_estimators': 843, 'max_depth': 82, 'learning_rate': 0.06609221478286383, 'subsample': 0.8438507748203137, 'colsample_bytree': 0.9767910810545337}. Best is trial 2 with value: 0.5846899721743519.
[I 2021-12-1

Best F1 score: 0.595012419877925
Best hyperparameters: {'n_estimators': 300, 'max_depth': 79, 'learning_rate': 0.04208439247743048, 'subsample': 0.4377450353973782, 'colsample_bytree': 0.2970209187817083}


### 4: Hyperparameter tuning for LightGBM + XGBoost (Hard Voting)

(tune most hyperparameters of LightGBM except scale_pos_weight(which is given the same value as before))

In [ ]:
def objective_SoftVoting(trial):
      n_estimators_values = trial.suggest_int('n_estimators', 10, 1000)
      max_depth_values = trial.suggest_int('max_depth', 1, 100)
      learning_rate_values = trial.suggest_float('learning_rate', 0.01, 0.3)
      subsample_values = trial.suggest_float('subsample', 0.2, 1)
      colsample_bytree_values = trial.suggest_float('colsample_bytree', 0.2, 1)
      clf1 = LGBMClassifier(scale_pos_weight=scale_pos_weight_data, 
                          n_estimators=n_estimators_values,
                          max_depth=max_depth_values,
                          learning_rate=learning_rate_values,
                          subsample=subsample_values,
                          colsample_bytree=colsample_bytree_values)
      clf2 = XGBClassifier(scale_pos_weight=scale_pos_weight_data)
      estimators=[]
      estimators.append(('lgbm', clf1))
      estimators.append(('xgb', clf2))
      clf = VotingClassifier(estimators, voting='hard')
      model_f1scores = cross_val_score(estimator = clf, X = X_train, y = Y_train, cv = k_fold, scoring = 'f1')
      return model_f1scores.mean()


study = optuna.create_study(direction='maximize')
study.optimize(objective_LightGBM, n_trials=100)

trial = study.best_trial
print('Best F1 score: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2021-12-10 07:26:31,002] A new study created in memory with name: no-name-3d69b552-d57d-4979-9e23-94faab26cf8d
[I 2021-12-10 07:26:41,341] Trial 0 finished with value: 0.5876234289654728 and parameters: {'n_estimators': 308, 'max_depth': 1, 'learning_rate': 0.205392081988844, 'subsample': 0.34403720302237994, 'colsample_bytree': 0.5304395478487254}. Best is trial 0 with value: 0.5876234289654728.
[I 2021-12-10 07:27:23,819] Trial 1 finished with value: 0.5915567496377735 and parameters: {'n_estimators': 478, 'max_depth': 10, 'learning_rate': 0.012257787043447078, 'subsample': 0.8244417984225176, 'colsample_bytree': 0.5202736229026061}. Best is trial 1 with value: 0.5915567496377735.
[I 2021-12-10 07:28:18,342] Trial 2 finished with value: 0.5867537850074582 and parameters: {'n_estimators': 679, 'max_depth': 74, 'learning_rate': 0.07327905465508809, 'subsample': 0.35224951836164237, 'colsample_bytree': 0.8619017734713907}. Best is trial 1 with value: 0.5915567496377735.
[I 2021-12-10

Best F1 score: 0.5949964005116721
Best hyperparameters: {'n_estimators': 623, 'max_depth': 95, 'learning_rate': 0.02784962418597878, 'subsample': 0.4699768259199653, 'colsample_bytree': 0.31085887604836576}


## (E) Reporting the final train and test scores for all 4 models(with tuned hyperparamaters)


In [ ]:
# Splitting into train and test sets 

X_train = X_train_unscaled
x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.2,shuffle=True,stratify=Y_train) 
# stratify=Y_train is used since its an imbalanced dataset. This parameter ensures that the data gets split according to the proportion of Y_train


# Scaling all columns
scaler = MinMaxScaler();
scaler.fit(x_train);
x_train = scaler.transform(x_train);
x_test = scaler.transform(x_test)

# =================  Model 1: LGBM with tuned hyperparameters: ===================================

# Best hyperparameters found out using hyperparameter tuning
best_hyperparameters = {'n_estimators': 760, 'max_depth': 85, 'learning_rate': 0.02935086260767057, 'subsample': 0.3792225838646886, 'colsample_bytree': 0.21433910546560286}
model = LGBMClassifier(**best_hyperparameters, scale_pos_weight=scale_pos_weight_data)
model.fit(x_train, y_train)
train_preds = model.predict(x_train)
test_preds = model.predict(x_test)

# Printing out accuracy, F1 score and balanced accuracy for train and test datasets
print("\nLGBM: ")
print("Train Accuracy:", accuracy_score(train_preds, y_train));
print("Test Accuracy:", accuracy_score(test_preds, y_test));
train_f1 = f1_score(train_preds, y_train);
test_f1 = f1_score(test_preds, y_test);
print("Train F1: {}".format(train_f1));
print("Test F1: {}".format(test_f1));
train_bac = balanced_accuracy_score(y_train, train_preds)
test_bac = balanced_accuracy_score(y_test, test_preds)
print("Train Balanced Accuracy: {}".format(train_bac))
print("Test Balanced Accuracy: {}".format(test_bac))

# =================  Model 2: XGBoost (tuning was not done): ===================================

# Best hyperparameters found out using hyperparameter tuning
model = XGBClassifier(scale_pos_weight=scale_pos_weight_data)
model.fit(x_train, y_train)
train_preds = model.predict(x_train)
test_preds = model.predict(x_test)

# Printing out accuracy, F1 score and balanced accuracy for train and test datasets
print("\nXGBoost: ")
print("Train Accuracy:", accuracy_score(train_preds, y_train));
print("Test Accuracy:", accuracy_score(test_preds, y_test));
train_f1 = f1_score(train_preds, y_train);
test_f1 = f1_score(test_preds, y_test);
print("Train F1: {}".format(train_f1));
print("Test F1: {}".format(test_f1));
train_bac = balanced_accuracy_score(y_train, train_preds)
test_bac = balanced_accuracy_score(y_test, test_preds)
print("Train Balanced Accuracy: {}".format(train_bac))
print("Test Balanced Accuracy: {}".format(test_bac))


# =================  Model 3: LGBM + XGBoost Soft Voting (tuning was done for hyperparameters of LGBM only): ===================================

estimators = []

# Best hyperparameters of LGBM in soft voting found out using hyperparameter tuning
best_hyperparameters = {'n_estimators': 300, 'max_depth': 79, 'learning_rate': 0.04208439247743048, 'subsample': 0.4377450353973782, 'colsample_bytree': 0.2970209187817083}
model_1 = LGBMClassifier(**best_hyperparameters,scale_pos_weight=scale_pos_weight_data)
estimators.append(('lgbm', model_1))

model_2 = XGBClassifier(scale_pos_weight=scale_pos_weight_data)
estimators.append(('xgb', model_2))

model = VotingClassifier(estimators, voting='soft')
model.fit(x_train, y_train)
train_preds = model.predict(x_train)
test_preds = model.predict(x_test)

# Printing out accuracy, F1 score and balanced accuracy for train and test datasets
print("\nLGBM + XGBoost Soft Voting: ")
print("Train Accuracy:", accuracy_score(train_preds, y_train));
print("Test Accuracy:", accuracy_score(test_preds, y_test));
train_f1 = f1_score(train_preds, y_train);
test_f1 = f1_score(test_preds, y_test);
print("Train F1: {}".format(train_f1));
print("Test F1: {}".format(test_f1));
train_bac = balanced_accuracy_score(y_train, train_preds)
test_bac = balanced_accuracy_score(y_test, test_preds)
print("Train Balanced Accuracy: {}".format(train_bac))
print("Test Balanced Accuracy: {}".format(test_bac))


# =================  Model 4: LGBM + XGBoost Hard Voting (tuning was done for hyperparameters of LGBM only): ===================================

estimators = []

# Best hyperparameters of LGBM in Hard voting found out using hyperparameter tuning
best_hyperparameters = {'n_estimators': 623, 'max_depth': 95, 'learning_rate': 0.02784962418597878, 'subsample': 0.4699768259199653, 'colsample_bytree': 0.31085887604836576}
model_1 = LGBMClassifier(**best_hyperparameters,scale_pos_weight=scale_pos_weight_data)
estimators.append(('lgbm', model_1))

model_2 = XGBClassifier(scale_pos_weight=scale_pos_weight_data)
estimators.append(('xgb', model_2))

model = VotingClassifier(estimators, voting='hard')
model.fit(x_train, y_train)
train_preds = model.predict(x_train)
test_preds = model.predict(x_test)

# Printing out accuracy, F1 score and balanced accuracy for train and test datasets
print("\nLGBM + XGBoost Hard Voting: ")
print("Train Accuracy:", accuracy_score(train_preds, y_train));
print("Test Accuracy:", accuracy_score(test_preds, y_test));
train_f1 = f1_score(train_preds, y_train);
test_f1 = f1_score(test_preds, y_test);
print("Train F1: {}".format(train_f1));
print("Test F1: {}".format(test_f1));
train_bac = balanced_accuracy_score(y_train, train_preds)
test_bac = balanced_accuracy_score(y_test, test_preds)
print("Train Balanced Accuracy: {}".format(train_bac))
print("Test Balanced Accuracy: {}".format(test_bac))


LGBM: 
Train Accuracy: 0.7735090361445783
Test Accuracy: 0.7292771084337349
Train F1: 0.6581967771994818
Test F1: 0.590486604702023
Train Balanced Accuracy: 0.7691061394699097
Test Balanced Accuracy: 0.7143084099681554

XGBoost: 
Train Accuracy: 0.7239006024096386
Test Accuracy: 0.7171686746987952
Train F1: 0.5981455908463208
Test F1: 0.5898488686992224
Train Balanced Accuracy: 0.7212286740669223
Test Balanced Accuracy: 0.7143165341922011

LGBM + XGBoost Soft Voting: 
Train Accuracy: 0.7410993975903615
Test Accuracy: 0.7232530120481928
Train F1: 0.6179523079316399
Test F1: 0.5924414478353441
Train Balanced Accuracy: 0.7373453449409979
Test Balanced Accuracy: 0.716272062384685

LGBM + XGBoost Hard Voting: 
Train Accuracy: 0.7594728915662651
Test Accuracy: 0.7337951807228915
Train F1: 0.62723771735325
Test F1: 0.5905679607152785
Train Balanced Accuracy: 0.7429525479400538
Test Balanced Accuracy: 0.7141644508443603


## (F) Predictions on the test dataset and final csv submission: Need to be completed

In [ ]:
## Training of the final model

In [ ]:
## Using the model trained above to make predictions on the test data

test_data_transformed = scaler.transform(test_data)
test_data_preds = model.predict(test_data_transformed)
cols = {'key':test_data['application_key'], 'prediction':test_data_preds};
submission_df = pd.DataFrame(cols)
submission_df

,key,prediction
0,578069,0
1,578070,0
2,578071,0
3,578072,0
4,578073,0
...,...,...
46995,310027,0
46996,310028,0
46997,310029,1
46998,310030,0


In [ ]:
submission_df.to_csv('Ida_group_20_9.csv', header=False, index=False)